Coursera의 ['Deep Learning Specialization'](https://www.coursera.org/specializations/deep-learning#about) 중 Improving Deep Neural Network : Hyperparameter tuning, Regularization and Optimization - Hpyerparameter tuning, Batch Normalization and Programing Framworks를 요약한 것

# Hyperparameter Tuning

Neural Network를 학습하는데 있어서 다양한 Hyperparameter를 세팅하는 과정이 개입되어있다.
* Learning Rate : $\alpha$
* Momentum Term : $\beta$
* Adam parameter : $\beta_1, \beta_2, \epsilon$
* No. of Layer, No. of Hidden Unit
* Mini-batch Size
* $\vdots$

Hyperparameter를 설정하고 고르는데 있어 특정 parameter가 더 특별히 중요할 수도 있다. 대부분의 경우에 Learning rate $\alpha$는 매우 중요한 Hyperparameter이다. Hyperparameter를 튜닝할 때에는 Hyperparameter들을 임의로 지정하여 시도해본다. 그 이유는 어떤 Hyperparameter 값이 문제에서 가장 중요한지 미리 알 수 없기 때문이다.
* Try random values(Not uniformly)
* Coarse to fine

## Using an appropriate scale to pick hyperparameters

Hyperparameter를 튜닝하기 위해 임의로 값을 설정할 때, 균일하게 임의로 샘플링 하는 것이 아니라, 적합한 scale을 골라서 Hyperparameter를 탐색해야 한다. 예를 들어 Learning rate $\alpha$를 튜닝할 때, 0.0001부터 1 사이를 균일하게 샘플링 한다면 90%가 0.1에서 1 사이에 있게 된다. 단지 10%만이 0.0001과 0.1 사이에 있기 때문에 좋은 탐색 방법이 아니다. 이런 경우에는 규일하게 샘플링 하는 대신 log scale을 사용하여 탐색을 하는 것이 더 좋은 방법이다.

# Batch Normalization